# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [2]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics

## 讀取資料

In [3]:
biopsy = dataset("MASS", "biopsy")

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32⍰,Int32,Int32,Int32,Categorical…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [53]:
df=biopsy[!, 2:6]
df=dropmissing(df)

,V1,V2,V3,V4,V5
,Int32,Int32,Int32,Int32,Int32
1,5,1,1,1,2
2,5,4,4,5,7
3,3,1,1,1,2
4,6,8,8,1,3
5,4,1,1,3,2
6,8,10,10,8,7
7,1,1,1,1,2
8,2,1,2,1,2
9,2,1,1,1,2


In [54]:
labels=Vector{String}(biopsy[!, :Class])
features=Matrix(df)

699×5 Array{Int32,2}:
 5   1   1  1  2
 5   4   4  5  7
 3   1   1  1  2
 6   8   8  1  3
 4   1   1  3  2
 8  10  10  8  7
 1   1   1  1  2
 2   1   2  1  2
 2   1   1  1  2
 4   2   1  1  2
 1   1   1  1  1
 2   1   1  1  2
 5   3   3  3  2
 ⋮              
 3   1   1  1  2
 4   1   1  1  2
 1   1   1  1  2
 1   1   1  3  2
 5  10  10  5  4
 3   1   1  1  2
 3   1   1  1  2
 3   1   1  1  3
 2   1   1  1  2
 5  10  10  3  7
 4   8   6  4  3
 4   8   8  5  4

In [55]:
model = DecisionTree.RandomForestClassifier(n_trees=50, max_depth=2)

RandomForestClassifier
n_trees:             50
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             nothing
ensemble:            nothing

In [56]:
DecisionTree.fit!(model, features, labels)

RandomForestClassifier
n_trees:             50
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             ["benign", "malignant"]
ensemble:            Ensemble of Decision Trees
Trees:      50
Avg Leaves: 4.0
Avg Depth:  2.0

In [58]:
using ScikitLearn.CrossValidation: cross_val_score

In [59]:
accuracy = cross_val_score(model, features, labels, cv=5)

5-element Array{Float64,1}:
 0.9148936170212766
 0.9357142857142857
 0.9214285714285714
 0.9640287769784173
 0.9568345323741008

In [61]:
new_biopsy = [5,4,4,5,7,10,3,2,1]
new_biopsy=convert(Array{Int32}, new_biopsy)

9-element Array{Int32,1}:
  5
  4
  4
  5
  7
 10
  3
  2
  1

In [64]:
DecisionTree.predict(model, new_biopsy)

"malignant"

In [63]:
DecisionTree.predict_proba(model, new_biopsy)

2-element Array{Float64,1}:
 0.04
 0.96